In [1]:

import os
os.chdir('..')


In [2]:
import torch
from torch import nn
import numpy as np
from torch.nn import functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from model.VAE import *

In [8]:
step = 1
batch_size = 5

device = 'cpu'

# Annealing parameter
k = 0.0025
x0 = 2500

loss_function = torch.nn.NLLLoss(reduction='sum')

model = VAE().to(device)

chord_onehot = torch.randn(batch_size, 272, 96)
length = torch.Tensor([272, 100, 200, 150, 120]).long()

chord_pred, logp, mu, log_var, input_x = model(chord_onehot,length)

# flatten tensor to calculate loss
# chord = torch.empty(batch_size, 272, 96 + 1, dtype=torch.long).random_(0, 1)

# Arrange 
chord_flatten = []
logp_flatten = []
length = length.squeeze()

for i in range(batch_size):
    # Get predicted softmax chords by length of the song (cutting off padding 0), (1,length,96)
    logp_flatten.append(logp[i][:length[i]])

    # Get groundtruth chords by length of the song (cutting off padding 0), (1,length)
    chord_flatten.append(chord_onehot[i][:length[i]])

# Rearrange for loss calculation
logp_flatten = torch.cat(logp_flatten, dim=0)
chord_flatten = torch.cat(chord_flatten,dim=0)
chord_groundtruth_index = torch.max(chord_flatten,1).indices

# loss calculation
# Add weight to NLL also
NLL_loss, KL_loss, KL_weight = loss_fn(loss_function = loss_function, logp = logp_flatten, target = chord_groundtruth_index, length = length, mean = mu, log_var = log_var,anneal_function='logistic', step=step, k=k, x0=x0)
# NLL_loss, KL_loss, KL_weight = loss_fn(logp = chord_pred_flatten, target = chord_groundtruth_index, length = length, mean = mu, log_var = log_var,anneal_function='logistic', step=step, k=k, x0=x0)

loss = (NLL_loss + KL_weight * KL_loss) / batch_size
print('loss: ',loss.item())


loss:  780.484130859375


In [2]:
import torch
from torch import nn
import numpy as np
from torch.nn import functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from model.Parameter_CVAE import *

In [3]:
step = 1
batch_size = 5

device = 'cpu'

# Annealing parameter
k = 0.0025
x0 = 2500

loss_function = torch.nn.NLLLoss(reduction='sum')

model = CVAE().to(device)

melody = torch.randn(batch_size, 272, 12 * 24 * 2)
chord_onehot = torch.randn(batch_size, 272, 96)
length = torch.Tensor([272, 100, 200, 150, 120]).long()
r_pitch = torch.Tensor([272, 100, 200, 150, 120]).view(batch_size,1,1).expand(batch_size,272,1)
r_rhythm = torch.Tensor([272, 100, 200, 150, 120]).view(batch_size,1,1).expand(batch_size,272,1)

z = torch.randn(batch_size,272,16)
z = torch.cat((z, melody, r_pitch, r_rhythm), dim=-1)

chord_pred,logp ,mu, log_var, input_x = model(chord_onehot,melody,length,r_pitch,r_rhythm)

# flatten tensor to calculate loss
# chord = torch.empty(batch_size, 272, 96 + 1, dtype=torch.long).random_(0, 1)

# Arrange 
chord_flatten = []
logp_flatten = []
length = length.squeeze()

for i in range(batch_size):
    # Get predicted softmax chords by length of the song (cutting off padding 0), (1,length,96)
    logp_flatten.append(logp[i][:length[i]])

    # Get groundtruth chords by length of the song (cutting off padding 0), (1,length)
    chord_flatten.append(chord_onehot[i][:length[i]])

# Rearrange for loss calculation
logp_flatten = torch.cat(logp_flatten, dim=0)
chord_flatten = torch.cat(chord_flatten,dim=0)
chord_groundtruth_index = torch.max(chord_flatten,1).indices

# loss calculation
# Add weight to NLL also
NLL_loss, KL_loss, KL_weight = loss_fn(loss_function = loss_function, logp = logp_flatten, target = chord_groundtruth_index, length = length, mean = mu, log_var = log_var,anneal_function='logistic', step=step, k=k, x0=x0)
# NLL_loss, KL_loss, KL_weight = loss_fn(logp = chord_pred_flatten, target = chord_groundtruth_index, length = length, mean = mu, log_var = log_var,anneal_function='logistic', step=step, k=k, x0=x0)

loss = (NLL_loss + KL_weight * KL_loss) / batch_size
print('loss: ',loss.item())


torch.Size([5, 272, 16])
loss:  1780.773681640625


In [4]:
z = torch.randn(batch_size,272,16)
z = torch.cat((z, melody,r_pitch), dim=-1)



In [35]:
z.shape

torch.Size([5, 272, 593])

In [7]:
print(melody.shape)

torch.Size([5, 272, 576])


In [8]:
print(z.shape)

torch.Size([5, 272, 592])


In [32]:
print(r_pitch.shape)

torch.Size([5, 272, 1])


In [10]:
a = torch.randn(batch_size)

In [11]:
a

tensor([-0.7862, -0.5978, -0.0300,  0.8419, -1.1588])

In [26]:
b = a.view(batch_size,1,1)
b

tensor([[[-0.7862]],

        [[-0.5978]],

        [[-0.0300]],

        [[ 0.8419]],

        [[-1.1588]]])

In [30]:
c = b.expand(5, 272, 1)
print(c.shape)

torch.Size([5, 272, 1])
